In [ ]:
# ==========================================
# FINAL FIXED PIPELINE (Robust Column Mapping)
# ==========================================
import os
import glob
import sqlite3
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from prettytable import PrettyTable

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from scipy.signal import medfilt
from scipy.stats import mode
from numpy.lib.stride_tricks import sliding_window_view
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

warnings.filterwarnings('ignore')

# --- 1. CONFIGURATION ---
class Parameters:
    def __init__(self):
        self.db_path, self.labels_path = self.find_kaggle_files()
        self.out_dir = "/kaggle/working/output/"
        self.sw_length = 1.0
        self.sampling_rate = 50
        self.sw_overlap = 50
        self.nb_channels = 3
        
        self.classes = {
            'aggressive posturing': 0, 'being mounted': 1, 'copulation attempt': 2, 
            'dynamic squatting': 3, 'flying': 4, 'foraging or drinking': 5,
            'mounting male': 6, 'other': 7, 'preening': 8, 'resting': 9, 
            'static squatting': 10, 'vigilance': 11, 'walking or running': 12
        }
        self.class_names = list(self.classes.keys())
        self.nb_classes = len(self.classes)
        
        self.cv_fold_assignments = {
            '1301': 0, '7-04-105': 1, '1361': 1, 'G20-0059-B6.5': 1,
            'G20-0529-B6.5': 2, '952': 3, '1331': 4, 'G20-0071-B6.5': 4,
            '1372': 5, '1326': 6, '1399': 7, '1681': 7, 'G20-0055-B6.5': 7,
            '1368': 8, '1333': 9
        }
        self.folds_to_run = sorted(list(set(self.cv_fold_assignments.values())))
        
        self.nb_filters = 128        
        self.filter_width = 11
        self.nb_units_lstm = 256
        self.nb_layers_lstm = 2      
        self.drop_prob = 0.4         
        self.batch_size = 256        
        self.epochs = 60             
        self.learning_rate = 1e-3    
        self.weight_decay = 1e-4
        self.gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.weighted = True
        self.train_excluding_transitions = True

    def find_kaggle_files(self):
        db_files = glob.glob("/kaggle/input/**/*.db", recursive=True)
        if not db_files: db_files = glob.glob("/kaggle/input/**/*.sqlite", recursive=True)
        csv_files = glob.glob("/kaggle/input/**/*.csv", recursive=True)
        # Find any CSV that looks like the labels file
        labels_path = next((f for f in csv_files if "behaviour" in f.lower()), csv_files[0] if csv_files else None)
        return db_files[0], labels_path

def seed_torch(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

def mkdir_if_missing(path):
    if not os.path.exists(path): os.makedirs(path)

# --- 2. ROBUST DATA LOADER ---
def load_dataset_joined(parameters):
    print(f"\n📂 Reading CSV: {parameters.labels_path}")
    labels_df = pd.read_csv(parameters.labels_path)
    
    # 1. Clean Columns
    rename_map = {
        'start_dt_real': 'start_time', 'Start (s)': 'start_time',
        'stop_dt_real':  'stop_time',  'Stop (s)':  'stop_time',
        'ruff_id': 'id', 'subject': 'id',
        'behaviour': 'behavior', 'recording_id': 'rec_id_str'
    }
    labels_df = labels_df.rename(columns=rename_map)
    
    # Auto-detect recording column if missing
    if 'rec_id_str' not in labels_df.columns:
        rec_col = next((c for c in labels_df.columns if 'recording' in c), None)
        if rec_col: labels_df.rename(columns={rec_col: 'rec_id_str'}, inplace=True)

    labels_df['start_time'] = pd.to_datetime(labels_df['start_time'])
    labels_df['stop_time'] = pd.to_datetime(labels_df['stop_time'])
    labels_df['id'] = labels_df['id'].astype(str).str.strip()

    conn = sqlite3.connect(parameters.db_path)
    
    # --- OPTIMIZATION: Pre-fetch Recording IDs ---
    print("🚀 Pre-fetching recording IDs to skip SQL JOINs...")
    rec_map_df = pd.read_sql_query("SELECT id, recording_id FROM recordings", conn)
    # Create dictionary: {'1372_L24_1': 1, ...}
    rec_map = dict(zip(rec_map_df['recording_id'], rec_map_df['id']))
    
    data_list = []
    print("⏳ Querying DB (Direct Integer Lookup)...")
    
    for _, row in tqdm(labels_df.iterrows(), total=len(labels_df)):
        rec_str = row['rec_id_str']
        
        # Get Integer ID from our map (Fast!)
        rec_int = rec_map.get(rec_str)
        if rec_int is None: continue # Skip if mapping missing
            
        start = str(row['start_time'])
        end = str(row['stop_time'])
        
        # OPTIMIZED QUERY: No JOIN, just direct integer lookup on Indexed column
        query = f"""
            SELECT datetime, accX, accY, accZ 
            FROM acc 
            WHERE recording_id = {rec_int} 
            AND datetime >= '{start}' AND datetime <= '{end}'
        """
        try:
            df = pd.read_sql_query(query, conn)
            if not df.empty:
                df['ruff_id'] = row['id']
                df['behaviour'] = row['behavior']
                data_list.append(df)
        except: continue
            
    conn.close()
    
    if not data_list: raise ValueError("No segments found!")
    full_data = pd.concat(data_list, ignore_index=True)
    
    clean_map = {k.strip(): v for k, v in parameters.cv_fold_assignments.items()}
    full_data = full_data.replace({'ruff_id': clean_map})
    full_data.rename(columns={'ruff_id':'fold_id'}, inplace=True)
    full_data.replace({'behaviour': parameters.classes}, inplace=True)
    full_data['behaviour'] = pd.to_numeric(full_data['behaviour'], errors='coerce').dropna().astype(int)
    full_data['fold_id'] = pd.to_numeric(full_data['fold_id'], errors='coerce').fillna(0).astype(int)
    
    print(f"🎉 Loaded {len(full_data)} rows.")
    return full_data
    
# --- 3. PREPROCESSING ---
def apply_sliding_window(full_data, parameters):
    w_len = int(parameters.sw_length * parameters.sampling_rate)
    stride = int(w_len * (1 - (parameters.sw_overlap / 100)))
    
    X_raw = full_data[['accX', 'accY', 'accZ']].values.astype(np.float32)
    y_raw = full_data['behaviour'].values.astype(np.int64)
    folds_raw = full_data['fold_id'].values.astype(np.int64)
    
    list_x, list_y, list_trans, list_fold = [], [], [], []
    print("Windowing...")
    for fold in np.unique(folds_raw):
        mask = (folds_raw == fold)
        curr_x, curr_y = X_raw[mask], y_raw[mask]
        if len(curr_x) < w_len: continue
        try:
            win_x = sliding_window_view(curr_x, window_shape=w_len, axis=0)[::stride]
            win_y = sliding_window_view(curr_y, window_shape=w_len, axis=0)[::stride]
        except: continue
        
        if hasattr(mode(win_y, axis=1), 'mode'): y_final = mode(win_y, axis=1).mode.flatten()
        else: y_final = mode(win_y, axis=1)[0].flatten()
        
        win_x = win_x.transpose(0, 2, 1)
        list_x.append(win_x); list_y.append(y_final)
        list_trans.append(mode(win_y, axis=1).count.flatten() != w_len)
        list_fold.append(np.full(len(y_final), fold))
        
    return {"win_X": np.concatenate(list_x), "win_y": np.concatenate(list_y), 
            "win_fold": np.concatenate(list_fold), "transition": np.concatenate(list_trans)}

# --- 4. DATASET & MODEL ---
class RuffDataset(Dataset):
    def __init__(self, x, y, augment=False):
        self.x = torch.from_numpy(x)
        self.y = torch.from_numpy(y).long()
        self.augment = augment
    def __len__(self): return len(self.x)
    def __getitem__(self, idx):
        x, y = self.x[idx], self.y[idx]
        if self.augment:
            angle = np.random.uniform(-np.pi/4, np.pi/4)
            rot = torch.tensor([[np.cos(angle), -np.sin(angle), 0], [np.sin(angle), np.cos(angle), 0], [0, 0, 1]], dtype=torch.float32)
            x = torch.matmul(x, rot) + torch.randn_like(x) * 0.02
        return x, y

class AttentionBlock(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attention = nn.Linear(hidden_size, 1)
    def forward(self, x):
        w = F.softmax(self.attention(x), dim=1)
        return torch.sum(w * x, dim=1)

class DeepConvLSTM_Attn(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 64, (5, 1)), nn.BatchNorm2d(64), nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, (5, 1)), nn.BatchNorm2d(64), nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(64, 128, (5, 1)), nn.BatchNorm2d(128), nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(128, 128, (5, 1)), nn.BatchNorm2d(128), nn.ReLU())
        self.lstm = nn.LSTM(128 * 3, config.nb_units_lstm, config.nb_layers_lstm, batch_first=True, dropout=0.2)
        self.attn = AttentionBlock(config.nb_units_lstm)
        self.fc = nn.Linear(config.nb_units_lstm, config.nb_classes)
        self.seq_len = 50 - 4 * 4

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv4(self.conv3(self.conv2(self.conv1(x))))
        x = x.permute(0, 2, 1, 3).reshape(x.size(0), self.seq_len, -1)
        x, _ = self.lstm(x)
        return self.fc(self.attn(x))

# --- 5. EXECUTION ---
if __name__ == "__main__":
    p = Parameters()
    mkdir_if_missing(p.out_dir)
    seed_torch(42)
    print(f"🚀 RUNNING PIPELINE | GPU: {p.gpu}")

    full_data = load_dataset_joined(p)
    data = apply_sliding_window(full_data, p)
    cv_scores, global_gt, global_preds = [], [], []
    
    print("\n=== STARTING LSIO ===")
    for fold in p.folds_to_run:
        print(f"\n>>> FOLD {fold}")
        
        train_mask = (data['win_fold'] != fold)
        if p.train_excluding_transitions: train_mask &= (~data['transition'])
        val_mask = (data['win_fold'] == fold)
        
        X_tr, y_tr = data['win_X'][train_mask], data['win_y'][train_mask]
        X_val, y_val = data['win_X'][val_mask], data['win_y'][val_mask]
        if len(X_val) == 0: continue
            
        train_ds = RuffDataset(X_tr, y_tr, augment=True)
        val_ds = RuffDataset(X_val, y_val, augment=False)
        train_l = DataLoader(train_ds, batch_size=p.batch_size, shuffle=True, num_workers=2)
        val_l = DataLoader(val_ds, batch_size=p.batch_size, shuffle=False, num_workers=2)
        
        net = DeepConvLSTM_Attn(p).to(p.gpu)
        w_full = np.ones(p.nb_classes)
        if p.weighted:
            w = compute_class_weight('balanced', classes=np.unique(y_tr), y=y_tr)
            w_full[np.unique(y_tr)] = w
        crit = nn.CrossEntropyLoss(weight=torch.FloatTensor(w_full).to(p.gpu))
        opt = torch.optim.AdamW(net.parameters(), lr=p.learning_rate, weight_decay=p.weight_decay)
        sched = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=p.learning_rate, steps_per_epoch=len(train_l), epochs=p.epochs)
        
        best_f1, best_preds = 0, None
        for ep in range(p.epochs):
            net.train()
            losses = []
            for x, y in train_l:
                x, y = x.to(p.gpu), y.to(p.gpu)
                opt.zero_grad()
                loss = crit(net(x), y)
                loss.backward()
                opt.step()
                sched.step()
                losses.append(loss.item())
                
            net.eval()
            preds, gt = [], []
            with torch.no_grad():
                for x, y in val_l:
                    x = x.to(p.gpu)
                    # TTA
                    out1 = torch.softmax(net(x), dim=1)
                    out2 = torch.softmax(net(x + torch.randn_like(x)*0.02), dim=1)
                    preds.extend(np.argmax((out1+out2).cpu().numpy(), axis=1))
                    gt.extend(y.numpy())
            
            f1 = f1_score(gt, preds, average='macro')
            if f1 > best_f1: best_f1, best_preds = f1, preds
            if ep % 5 == 0: print(f"Ep {ep}: Loss {np.mean(losses):.4f} | Val F1 {f1:.4f}")

        smoothed = medfilt(best_preds, kernel_size=5)
        final_f1 = f1_score(y_val, smoothed, average='macro')
        cv_scores.append(final_f1)
        print(f"FOLD {fold} FINAL F1: {final_f1:.4f}")
        
        global_gt.extend(y_val)
        global_preds.extend(smoothed)
        
        # Save confusion matrix plot
        cm = confusion_matrix(y_val, smoothed)
        plt.figure(figsize=(8,6))
        sns.heatmap(cm, cmap='Blues'); plt.title(f"Fold {fold}"); plt.savefig(f"{p.out_dir}/cm_{fold}.png"); plt.close()

    print(f"\n=== MEAN F1: {np.mean(cv_scores):.4f} ===")
    print(classification_report(global_gt, global_preds, target_names=p.class_names))

In [1]:
# ==========================================
# FINAL "ULTRA-OPTIMIZED" PIPELINE
# Combines: Memory-Optimized Loading + Advanced Augmentation/TTA
# ==========================================
import os
import glob
import gc
import sqlite3
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from prettytable import PrettyTable

# Torch & Sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from scipy.signal import medfilt
from scipy.stats import mode
from numpy.lib.stride_tricks import sliding_window_view
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

warnings.filterwarnings('ignore')

# ==========================================
# 1. CONFIGURATION
# ==========================================
class Parameters:
    def __init__(self):
        self.db_path, self.labels_path = self.find_kaggle_files()
        self.out_dir = "/kaggle/working/output/"
        
        # Data Config
        self.sw_length = 1.0
        self.sampling_rate = 50
        self.sw_overlap = 50
        self.nb_channels = 3
        
        # Class Map
        self.classes = {
            'aggressive posturing': 0, 'being mounted': 1, 'copulation attempt': 2, 
            'dynamic squatting': 3, 'flying': 4, 'foraging or drinking': 5,
            'mounting male': 6, 'other': 7, 'preening': 8, 'resting': 9, 
            'static squatting': 10, 'vigilance': 11, 'walking or running': 12
        }
        self.class_names = list(self.classes.keys())
        self.nb_classes = len(self.classes)
        
        # 10-Fold Assignment
        self.cv_fold_assignments = {
            '1301': 0, '7-04-105': 1, '1361': 1, 'G20-0059-B6.5': 1,
            'G20-0529-B6.5': 2, '952': 3, '1331': 4, 'G20-0071-B6.5': 4,
            '1372': 5, '1326': 6, '1399': 7, '1681': 7, 'G20-0055-B6.5': 7,
            '1368': 8, '1333': 9
        }
        self.folds_to_run = sorted(list(set(self.cv_fold_assignments.values())))
        
        # Optimized Hyperparameters
        self.nb_filters = 64
        self.filter_width = 11
        self.nb_units_lstm = 256
        self.nb_layers_lstm = 2      # Increased Depth for better representation
        self.drop_prob = 0.5
        self.batch_size = 512
        self.epochs = 60             # Sufficient for OneCycleLR
        self.learning_rate = 1e-3
        self.weight_decay = 1e-5
        self.gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Flags
        self.weighted = True
        self.train_excluding_transitions = True

    def find_kaggle_files(self):
        print("🔎 Searching for data...")
        db_files = glob.glob("/kaggle/input/**/*.db", recursive=True)
        if not db_files: db_files = glob.glob("/kaggle/input/**/*.sqlite", recursive=True)
        csv_files = glob.glob("/kaggle/input/**/*.csv", recursive=True)
        labels_path = next((f for f in csv_files if "behaviour" in f.lower()), csv_files[0] if csv_files else None)
        return db_files[0], labels_path

def seed_torch(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

def mkdir_if_missing(path):
    if not os.path.exists(path): os.makedirs(path)


# ==========================================
# 2. DATA PROCESSING (DEBUGGED & FIXED)
# ==========================================

def create_dataset_optimized(parameters):
    print(f"\n📂 Reading CSV: {parameters.labels_path}")
    labels_df = pd.read_csv(parameters.labels_path)
    
    # 1. Normalize Columns
    labels_df.columns = [c.strip().lower() for c in labels_df.columns]
    print(f"   Raw Columns: {labels_df.columns.tolist()}")
    
    col_map = {}
    
    # --- STRICT PRIORITY MAPPING ---
    # We look for the best match first. If found, we stop looking.
    
    # ID Column Priorities
    id_candidates = ['ruff_id', 'individual_id', 'subject', 'id']
    for c in id_candidates:
        if c in labels_df.columns:
            col_map[c] = 'id'
            break
            
    # Behavior Column Priorities
    beh_candidates = ['behaviour', 'behavior', 'activity']
    for c in beh_candidates:
        if c in labels_df.columns:
            col_map[c] = 'behavior'
            break
            
    # Start Time Priorities (Crucial Fix: Prioritize 'real' over 'video')
    start_candidates = ['start_dt_real', 'start (s)', 'start_time', 'start']
    for c in start_candidates:
        if c in labels_df.columns:
            col_map[c] = 'start_time'
            break
            
    # Stop Time Priorities
    stop_candidates = ['stop_dt_real', 'stop (s)', 'stop_time', 'stop', 'end']
    for c in stop_candidates:
        if c in labels_df.columns:
            col_map[c] = 'stop_time'
            break
            
    # Recording ID Priorities
    rec_candidates = ['recording_id', 'rec_id', 'filename']
    for c in rec_candidates:
        if c in labels_df.columns:
            col_map[c] = 'rec_id_str'
            break
    
    print(f"   Mapping Applied: {col_map}")
    
    # Apply Rename
    labels_df = labels_df.rename(columns=col_map)
    
    # Keep ONLY the columns we need (Drops duplicates/unused)
    req_cols = ['id', 'behavior', 'start_time', 'stop_time', 'rec_id_str']
    
    # check missing
    missing = [rc for rc in req_cols if rc not in labels_df.columns]
    if missing:
        raise KeyError(f"Missing columns after mapping: {missing}")
        
    # Filter DataFrame to just these 5 columns to prevent duplicates
    labels_df = labels_df[req_cols].copy()

    # 2. Convert Types
    labels_df['start_time'] = pd.to_datetime(labels_df['start_time'])
    labels_df['stop_time'] = pd.to_datetime(labels_df['stop_time'])
    labels_df['rec_id_str'] = labels_df['rec_id_str'].astype(str).str.strip()
    labels_df['id'] = labels_df['id'].astype(str).str.strip()

    # 3. Prepare Mappings
    fold_map = {k.strip(): v for k, v in parameters.cv_fold_assignments.items()}
    
    conn = sqlite3.connect(parameters.db_path)
    try:
        rec_map_df = pd.read_sql_query("SELECT id, recording_id FROM recordings", conn)
        rec_int_map = dict(zip(rec_map_df['recording_id'], rec_map_df['id']))
    except:
        raise ValueError("Could not map recordings! Check DB schema.")
    
    # 4. Setup Windowing
    w_len = int(parameters.sw_length * parameters.sampling_rate)
    stride = int(w_len * (1 - (parameters.sw_overlap / 100)))
    
    master_X, master_y, master_fold, master_trans = [], [], [], []
    
    grouped = labels_df.groupby('rec_id_str')
    print(f"🚀 Processing {len(grouped)} recordings...")
    
    for rec_str, group in tqdm(grouped):
        rec_int = rec_int_map.get(rec_str)
        if rec_int is None: continue
            
        min_t = group['start_time'].min()
        max_t = group['stop_time'].max()
        
        query = f"""
            SELECT datetime, accX, accY, accZ 
            FROM acc 
            WHERE recording_id = {rec_int} 
            AND datetime >= '{min_t}' AND datetime <= '{max_t}'
        """
        try:
            rec_df = pd.read_sql_query(query, conn)
            if rec_df.empty: continue
            rec_df['datetime'] = pd.to_datetime(rec_df['datetime'])
            rec_df = rec_df.sort_values('datetime')
        except: continue
            
        for _, row in group.iterrows():
            mask = (rec_df['datetime'] >= row['start_time']) & (rec_df['datetime'] <= row['stop_time'])
            segment = rec_df[mask]
            
            if len(segment) < w_len: continue
                
            x_raw = segment[['accX', 'accY', 'accZ']].values.astype(np.float32)
            bird_id = row['id']
            fold_id = fold_map.get(bird_id, -1)
            label = parameters.classes.get(row['behavior'], -1)
            
            if fold_id == -1 or label == -1: continue
            
            try:
                wins = sliding_window_view(x_raw, window_shape=w_len, axis=0)[::stride]
                wins = wins.transpose(0, 2, 1) # (N, 50, 3)
                
                n = len(wins)
                master_X.append(wins)
                master_y.append(np.full(n, label, dtype=np.int64))
                master_fold.append(np.full(n, fold_id, dtype=np.int64))
                master_trans.append(np.zeros(n, dtype=bool))
            except: continue
        
        del rec_df
        gc.collect()

    conn.close()
    
    if not master_X: raise ValueError("No data created! Check mappings.")
    
    print("Concatenating...")
    return {
        "win_X": np.concatenate(master_X, axis=0),
        "win_y": np.concatenate(master_y, axis=0),
        "win_fold": np.concatenate(master_fold, axis=0),
        "transition": np.concatenate(master_trans, axis=0)
    }
    
# ==========================================
# 3. DATASET WITH AUGMENTATION
# ==========================================
class RuffDataset(Dataset):
    def __init__(self, x, y, augment=False):
        self.x = torch.from_numpy(x)
        self.y = torch.from_numpy(y).long()
        self.augment = augment

    def __len__(self): return len(self.x)

    def __getitem__(self, idx):
        x, y = self.x[idx], self.y[idx]
        if self.augment:
            # Random Rotation
            angle = np.random.uniform(-np.pi/6, np.pi/6)
            c, s = np.cos(angle), np.sin(angle)
            rot = torch.tensor([[c, -s, 0], [s, c, 0], [0, 0, 1]], dtype=torch.float32)
            x = torch.matmul(x, rot)
            # Noise
            x += torch.randn_like(x) * 0.01
        return x, y

# ==========================================
# 4. MODEL (DeepConvLSTM+)
# ==========================================
class DeepConvLSTM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.nb_filters = config.nb_filters
        self.nb_classes = config.nb_classes
        
        # 2 Conv Blocks + Batch Norm
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, (11, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, (11, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        
        self.final_seq_len = 50 - (11-1)*2
        self.lstm = nn.LSTM(64*3, config.nb_units_lstm, config.nb_layers_lstm, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(config.nb_units_lstm, self.nb_classes)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv2(self.conv1(x))
        x = x.permute(0, 2, 1, 3).reshape(x.size(0), -1, 64*3)
        x, _ = self.lstm(x)
        return self.fc(x[:, -1, :])

def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)): nn.init.xavier_normal_(m.weight)
    if isinstance(m, nn.LSTM): 
        for n, p in m.named_parameters():
            if 'weight' in n: nn.init.xavier_normal_(p)

# ==========================================
# 5. VISUALIZATION UTILS
# ==========================================
def plot_metrics(history, fold, out_dir):
    plt.figure(figsize=(10,5))
    plt.plot(history['loss'], label='Train Loss', color='red')
    plt.twinx().plot(history['f1'], label='Val F1', color='blue')
    plt.title(f"Fold {fold} Training")
    plt.legend()
    plt.savefig(f"{out_dir}/curve_{fold}.png")
    plt.close()

# ==========================================
# 6. MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    p = Parameters()
    mkdir_if_missing(p.out_dir)
    seed_torch(42)
    print(f"🚀 RUNNING OPTIMIZED PIPELINE | GPU: {p.gpu}")

    # 1. Create Data
    data = create_dataset_optimized(p)
    print(f"✅ Dataset Ready: {len(data['win_y'])} windows")
    
    cv_scores, global_gt, global_preds = [], [], []
    
    print("\n=== STARTING LSIO ===")
    for fold in p.folds_to_run:
        print(f"\n>>> FOLD {fold}")
        
        train_mask = (data['win_fold'] != fold)
        if p.train_excluding_transitions: train_mask &= (~data['transition'])
        val_mask = (data['win_fold'] == fold)
        
        X_tr, y_tr = data['win_X'][train_mask], data['win_y'][train_mask]
        X_val, y_val = data['win_X'][val_mask], data['win_y'][val_mask]
        
        if len(X_val) == 0: continue
            
        # Augmentation on Train, No Aug on Val
        train_ds = RuffDataset(X_tr, y_tr, augment=True)
        val_ds = RuffDataset(X_val, y_val, augment=False)
        
        train_l = DataLoader(train_ds, batch_size=p.batch_size, shuffle=True, num_workers=2)
        val_l = DataLoader(val_ds, batch_size=p.batch_size, shuffle=False, num_workers=2)
        
        net = DeepConvLSTM(p).to(p.gpu)
        net.apply(init_weights)
        
        # Weighted Loss
        w_full = np.ones(p.nb_classes)
        if p.weighted and len(y_tr) > 0:
            w = compute_class_weight('balanced', classes=np.unique(y_tr), y=y_tr)
            w_full[np.unique(y_tr)] = w
        crit = nn.CrossEntropyLoss(weight=torch.FloatTensor(w_full).to(p.gpu))
        
        opt = torch.optim.AdamW(net.parameters(), lr=p.learning_rate, weight_decay=p.weight_decay)
        sched = torch.optim.lr_scheduler.OneCycleLR(opt, max_lr=p.learning_rate, steps_per_epoch=len(train_l), epochs=p.epochs)
        
        best_f1, best_preds = 0, None
        history = {'loss': [], 'f1': []}
        
        for ep in range(p.epochs):
            net.train()
            losses = []
            for x, y in train_l:
                x, y = x.to(p.gpu), y.to(p.gpu)
                opt.zero_grad()
                loss = crit(net(x), y)
                loss.backward()
                opt.step()
                sched.step()
                losses.append(loss.item())
                
            # TTA VALIDATION
            net.eval()
            preds, gt = [], []
            with torch.no_grad():
                for x, y in val_l:
                    x = x.to(p.gpu)
                    # TTA: Normal + Noisy
                    out1 = torch.softmax(net(x), dim=1)
                    out2 = torch.softmax(net(x + torch.randn_like(x)*0.01), dim=1)
                    avg = (out1 + out2) / 2
                    preds.extend(np.argmax(avg.cpu().numpy(), axis=1))
                    gt.extend(y.numpy())
            
            f1 = f1_score(gt, preds, average='macro')
            history['loss'].append(np.mean(losses))
            history['f1'].append(f1)
            
            if f1 > best_f1: best_f1, best_preds = f1, preds
            if ep % 10 == 0: print(f"Ep {ep}: Loss {np.mean(losses):.4f} | Val F1 {f1:.4f}")

        # Smoothing
        smoothed = medfilt(best_preds, kernel_size=5)
        final_f1 = f1_score(y_val, smoothed, average='macro')
        cv_scores.append(final_f1)
        print(f"FOLD {fold} FINAL F1: {final_f1:.4f}")
        
        # Store & Save
        global_gt.extend(y_val)
        global_preds.extend(smoothed)
        plot_metrics(history, fold, p.out_dir)
        
        cm = confusion_matrix(y_val, smoothed)
        plt.figure(figsize=(8,6))
        sns.heatmap(cm, cmap='Blues'); plt.title(f"Fold {fold}"); plt.savefig(f"{p.out_dir}/cm_{fold}.png"); plt.close()

    print(f"\n=== MEAN F1: {np.mean(cv_scores):.4f} ===")
    print(classification_report(global_gt, global_preds, target_names=p.class_names))

🔎 Searching for data...
🚀 RUNNING OPTIMIZED PIPELINE | GPU: cuda

📂 Reading CSV: /kaggle/input/ruff-behaviour-data/ruff_behaviour_tidy_2022-11-16.csv
   Raw Columns: ['beh_event_id', 'recording_id', 'video_analysis_id', 'ruff_id', 'ruff_name', 'morph', 'video_file_start_dt', 'video_start_secs', 'video_stop_secs', 'start_dt_real', 'stop_dt_real', 'duration_secs', 'behaviour']
   Mapping Applied: {'ruff_id': 'id', 'behaviour': 'behavior', 'start_dt_real': 'start_time', 'stop_dt_real': 'stop_time', 'recording_id': 'rec_id_str'}
🚀 Processing 28 recordings...


  0%|          | 0/28 [00:00<?, ?it/s]

Concatenating...
✅ Dataset Ready: 136622 windows

=== STARTING LSIO ===

>>> FOLD 0
Ep 0: Loss 2.1102 | Val F1 0.1946
Ep 10: Loss 1.2558 | Val F1 0.3854
Ep 20: Loss 1.0270 | Val F1 0.2713
Ep 30: Loss 0.8152 | Val F1 0.3437
Ep 40: Loss 0.6145 | Val F1 0.4028
Ep 50: Loss 0.5081 | Val F1 0.3891
FOLD 0 FINAL F1: 0.4287

>>> FOLD 1
Ep 0: Loss 2.1218 | Val F1 0.2224
Ep 10: Loss 1.2380 | Val F1 0.2700
Ep 20: Loss 1.0314 | Val F1 0.2948
Ep 30: Loss 0.7481 | Val F1 0.3648
Ep 40: Loss 0.5954 | Val F1 0.3692
Ep 50: Loss 0.4841 | Val F1 0.3490
FOLD 1 FINAL F1: 0.4022

>>> FOLD 2
Ep 0: Loss 2.0996 | Val F1 0.2679
Ep 10: Loss 1.2169 | Val F1 0.2836
Ep 20: Loss 0.9761 | Val F1 0.3501
Ep 30: Loss 0.7453 | Val F1 0.3294
Ep 40: Loss 0.5574 | Val F1 0.3438
Ep 50: Loss 0.4481 | Val F1 0.3594
FOLD 2 FINAL F1: 0.3653

>>> FOLD 3
Ep 0: Loss 2.1185 | Val F1 0.2523
Ep 10: Loss 1.2086 | Val F1 0.3056
Ep 20: Loss 0.9718 | Val F1 0.3542
Ep 30: Loss 0.7788 | Val F1 0.3202
Ep 40: Loss 0.5912 | Val F1 0.3359
Ep 50: 